In [36]:
import copy
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import pickle
import lightgbm as lgb
import category_encoders as ce

In [37]:
# Load data
df = pd.read_csv('../data/data_preprocessed.csv')

In [38]:
df.head()

,year,price,assemble_place,series,km,transmission,brand,model,engine_type_Dầu,engine_type_Hybrid,engine_type_Xăng,engine_type_Điện
0,2020.0,8.990000e+08,0,1.340081e+09,60000,1,6.027708e+08,1.009069e+09,True,False,False,False
1,2018.0,5.680000e+08,0,1.340081e+09,80000,1,6.027708e+08,5.025594e+08,False,False,True,False
2,2022.0,9.290000e+08,0,1.340081e+09,50000,1,6.174147e+08,9.519027e+08,False,False,True,False
3,2022.0,3.690000e+08,1,8.902675e+08,25000,0,4.756669e+08,3.770852e+08,False,False,True,False
4,2011.0,4.900000e+09,1,8.902675e+08,80000,1,1.625612e+09,4.775661e+09,False,False,True,False


In [39]:
# price is the target variable
df.isna().sum()

year                  0
price                 0
assemble_place        0
series                0
km                    0
transmission          0
brand                 0
model                 0
engine_type_Dầu       0
engine_type_Hybrid    0
engine_type_Xăng      0
engine_type_Điện      0
dtype: int64

In [40]:
X = df.drop('price', axis=1)
y = df['price']

In [41]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
# Linear Regression
regressor = LinearRegression()
regressor.fit(x_train, y_train)

LinearRegression()

In [43]:
regressor.score(x_test, y_test)

0.898823445627658

In [44]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100)
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [45]:
from sklearn.model_selection import KFold, RandomizedSearchCV

"""
+ Search over space of hyperparameters to find acceptable ones
+ Two search methods implemented by scikit-learn: GridSearch and RandomSearch
+ RandomSearch is faster in case we search over large space (too many combinations of hyperparameters)
"""

rf_hyperparams = {
    'n_estimators': [200, 300, 400, 500],
    'max_depth': [None, 10, 50, 90, 110],
    'max_features': ['sqrt'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

rf_randomized_search = RandomizedSearchCV(
    estimator = rf,
    param_distributions = rf_hyperparams,
    n_iter = 10,
    cv=KFold(n_splits=10, shuffle=True, random_state=42),
    verbose=2,
    random_state=42,
    n_jobs = -1
)



In [46]:
rf_randomized_search.fit(x_train, y_train)


Fitting 10 folds for each of 10 candidates, totalling 100 fits


RandomizedSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
                   estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 10, 50, 90, 110],
                                        'max_features': ['sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 300, 400, 500]},
                   random_state=42, verbose=2)

In [47]:
rf_randomized_search.score(x_test, y_test)


0.9699292086516738

In [48]:
lgbm = lgb.LGBMRegressor()
lgbm.fit(x_train, y_train)

y_pred = lgbm.predict(x_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
#score
print("score: ",lgbm.score(x_test,y_test))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001083 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 583
[LightGBM] [Info] Number of data points in the train set: 9511, number of used features: 11
[LightGBM] [Info] Start training from score 1015258757.080433
MAE: 71751267.2501273
MSE: 2.7002449847376816e+16
RMSE: 164324221.73062867
score:  0.9731205753856111


In [49]:
#Gradient Boosting Regressor
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor()
gbr.fit(x_train, y_train)
y_pred = gbr.predict(x_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
#score
print("score: ",gbr.score(x_test,y_test))

MAE: 105307526.43301873
MSE: 5.1748886402460536e+16
RMSE: 227483815.6934698
score:  0.9484868854938865


In [50]:
# XGBoost Regressor
from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(x_train, y_train)
y_pred = xgb.predict(x_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
#score
print("score: ",xgb.score(x_test,y_test))


MAE: 61471665.3961312
MSE: 2.888323326728325e+16
RMSE: 169950678.92563197
score:  0.9712483609592495


In [51]:
# in ra bảng so sánh điểm,MAE,MSE,RMSE của các model
models = ['Linear Regression', 'Random Forest', 'LightGBM', 'Gradient Boosting', 'XGBoost']
scores = [regressor.score(x_test, y_test), rf_randomized_search.score(x_test, y_test), lgbm.score(x_test, y_test), gbr.score(x_test, y_test), xgb.score(x_test, y_test)]
mae = [metrics.mean_absolute_error(y_test, regressor.predict(x_test)), metrics.mean_absolute_error(y_test, rf_randomized_search.predict(x_test)), metrics.mean_absolute_error(y_test, lgbm.predict(x_test)), metrics.mean_absolute_error(y_test, gbr.predict(x_test)), metrics.mean_absolute_error(y_test, xgb.predict(x_test))]
mse = [metrics.mean_squared_error(y_test, regressor.predict(x_test)), metrics.mean_squared_error(y_test, rf_randomized_search.predict(x_test)), metrics.mean_squared_error(y_test, lgbm.predict(x_test)), metrics.mean_squared_error(y_test, gbr.predict(x_test)), metrics.mean_squared_error(y_test, xgb.predict(x_test))]
rmse = [np.sqrt(metrics.mean_squared_error(y_test, regressor.predict(x_test))), np.sqrt(metrics.mean_squared_error(y_test, rf_randomized_search.predict(x_test))), np.sqrt(metrics.mean_squared_error(y_test, lgbm.predict(x_test))), np.sqrt(metrics.mean_squared_error(y_test, gbr.predict(x_test))), np.sqrt(metrics.mean_squared_error(y_test, xgb.predict(x_test)))]
c_df = pd.DataFrame({'Model': models, 'Score': scores, 'MAE': mae, 'MSE': mse, 'RMSE': rmse})
print(c_df)

               Model     Score           MAE           MSE          RMSE
0  Linear Regression  0.898823  1.668024e+08  1.016396e+17  3.188097e+08
1      Random Forest  0.969929  6.245602e+07  3.020842e+16  1.738057e+08
2           LightGBM  0.973121  7.175127e+07  2.700245e+16  1.643242e+08
3  Gradient Boosting  0.948487  1.053075e+08  5.174889e+16  2.274838e+08
4            XGBoost  0.971248  6.147167e+07  2.888323e+16  1.699507e+08


In [52]:
# save model
filename = 'finalized_model.pkl'
pickle.dump(xgb, open(filename, 'wb'))